In [1]:
import pandas as pd
import numpy as np
import tgt
import re
import os
import pathlib
import utils

In [2]:
# Read the TextGrid
def read_tgt(session):

    filepath = 'transcriptions_annotations/' + session

    tg_path = filepath + '/' + session + '.TextGrid'
    tg = tgt.read_textgrid(tg_path) # my text grid files
    print(tg_path)
    return tg 

In [3]:
def load_session_annotation(doc_no):
    tg = read_tgt(doc_no)
    return tg    

In [4]:
def get_session_start_end(tg):
    
    part = tg.get_tier_by_name('Part')
    part_intervals = part.intervals
    part.end_time

    session_start = part.intervals[0].start_time 
    session_end = part.intervals[0].end_time
    
    return { 'session_start': session_start, 'session_end': session_end } 

In [5]:
def load_participant_annotation(sess, participant):
    # works well 
    session_id = int(sess[-1:]) ## you have to change this to make it work for double digits !!!
    participant_id = str(session_id) +'_' + participant
    participant_utts = str(participant) + '-utts' 
    
    session_annotation = load_session_annotation(sess)
    session_start_end = get_session_start_end(session_annotation)
    session_start = float(session_start_end['session_start'])
    session_end = float(session_start_end['session_end'])
    
    participant_annotation = session_annotation.get_tier_by_name(participant_utts)
    participant_annotation.delete_annotations_with_text(pattern='laughter', n=0, regex=True) # filter laughter
    participant_partials = participant_annotation.get_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
    
    # nested dictionary or nested list
    
    partials = {} 
    fluents = {}
    
    d = {
        'session_id': session_id,
        'session_start': session_start,
        'session_end': session_end,
        'participant_id': participant_id,
        'partials': partials,
        'fluents': fluents
    }
    
    for i, interval in enumerate(participant_partials):
        # if interval.start_time in np.arange(session_start, session_end):
        start_time = interval.start_time
        # if interval.end_time in np.arange(session_start, session_end): # does not work perhaps for i, in ranger
        end_time = interval.end_time 
        partial_no = i
        disfluency = 1 
        speaker = 1
        partials[i] = {'speaker':speaker, 'partial_no': partial_no, 'disfluency':disfluency, 'start_time': start_time, 'end_time': end_time}
        
    participant_annotation.delete_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
    participant_annotation.delete_annotations_with_text(pattern='<p=', n=0, regex=True)
    participant_annotation.delete_annotations_with_text(pattern='breathing', n=0, regex=True)
    participant_fluent_intervals = participant_annotation
    
    for j, interval_flu in enumerate(participant_fluent_intervals):
        start_time = interval_flu.start_time
        end_time = interval_flu.end_time 
        fluent_no = j
        disfluency = 0 
        speaker = 1
        fluents[j] = {'speaker':speaker, 'fluent_no': fluent_no, 'disfluency':disfluency, 'start_time': start_time, 'end_time': end_time}
        
    return d

In [28]:
all_sessions = {1: 'r1', 2: 'r2', 3:'r3', 4:'r4', 5:'r5', 6:'r6', 7:'r7', 8:'r8', 9:'r9'}
# apply to : 10: 'r10', 11: 'r11', 12: 'r12', 13: 'r13', 14: 'r14', 15: 'r15', 16: 'r16', 17: 'r17', 18: 'r18', 19: 'r19'
participants = {'A', 'B'}

dict_all = []
for sessions in all_sessions:
    for participant in participants:
        dict_all.append(load_participant_annotation(all_sessions[int(sessions)], participant))
        

transcriptions_annotations/r1/r1.TextGrid
transcriptions_annotations/r1/r1.TextGrid
transcriptions_annotations/r2/r2.TextGrid
transcriptions_annotations/r2/r2.TextGrid
transcriptions_annotations/r3/r3.TextGrid
transcriptions_annotations/r3/r3.TextGrid
transcriptions_annotations/r4/r4.TextGrid
transcriptions_annotations/r4/r4.TextGrid
transcriptions_annotations/r5/r5.TextGrid
transcriptions_annotations/r5/r5.TextGrid
transcriptions_annotations/r6/r6.TextGrid
transcriptions_annotations/r6/r6.TextGrid
transcriptions_annotations/r7/r7.TextGrid
transcriptions_annotations/r7/r7.TextGrid
transcriptions_annotations/r8/r8.TextGrid
transcriptions_annotations/r8/r8.TextGrid
transcriptions_annotations/r9/r9.TextGrid
transcriptions_annotations/r9/r9.TextGrid
transcriptions_annotations/r10/r10.TextGrid
transcriptions_annotations/r10/r10.TextGrid
transcriptions_annotations/r11/r11.TextGrid
transcriptions_annotations/r11/r11.TextGrid
transcriptions_annotations/r12/r12.TextGrid
transcriptions_annotatio

[{'session_id': 1,
  'session_start': 547.168380434,
  'session_end': 1321.59253469,
  'participant_id': '1_A',
  'partials': {0: {'speaker': 1,
    'partial_no': 0,
    'disfluency': 1,
    'start_time': 605.446997075,
    'end_time': 607.403301579},
   1: {'speaker': 1,
    'partial_no': 1,
    'disfluency': 1,
    'start_time': 640.866221459,
    'end_time': 643.835303074},
   2: {'speaker': 1,
    'partial_no': 2,
    'disfluency': 1,
    'start_time': 659.644876156,
    'end_time': 662.754119798},
   3: {'speaker': 1,
    'partial_no': 3,
    'disfluency': 1,
    'start_time': 669.377786575,
    'end_time': 681.827085792},
   4: {'speaker': 1,
    'partial_no': 4,
    'disfluency': 1,
    'start_time': 691.41017797,
    'end_time': 693.80794942},
   5: {'speaker': 1,
    'partial_no': 5,
    'disfluency': 1,
    'start_time': 722.804262555,
    'end_time': 723.47079108},
   6: {'speaker': 1,
    'partial_no': 6,
    'disfluency': 1,
    'start_time': 750.476174853,
    'end_time':

In [25]:
all_records_disfluency_labels = []
all_records_fluent_labels = []

for i, participant in enumerate(dict_all):
    participant_id = dict_all[i]['participant_id']

        
    partials = dict_all[i]['partials']
    partial_count = 0
    
    for partial in partials:
        start = partials[partial]['start_time']
        end = partials[partial]['end_time']
        partial_no = partials[partial]['partial_no']
        disfluency = 1
        partial_count += 1
        all_records_disfluency_labels.append([disfluency, participant_id, partial_no, start, end])
     
    fluents = dict_all[i]['fluents']
    fluent_count = 0
    
    for fluent in fluents:
        start2 = fluents[fluent]['start_time']
        end2 = fluents[fluent]['end_time']
        fluent_no = fluents[fluent]['fluent_no']         
        disfluency = 0
        fluent_count += 1
        all_records_fluent_labels.append([disfluency, participant_id, fluent_no, start2, end2]) 

                
all_records_disfluency_dataframe = pd.DataFrame(all_records_disfluency_labels, columns = ['Disfluency', 'ParticipantID', 'EventNo', 'Start', 'End'])
all_records_fluent_dataframe = pd.DataFrame(all_records_fluent_labels, columns = ['Disfluency', 'ParticipantID', 'EventNo', 'Start', 'End'])

1_A
1_B
2_A
2_B
3_A
3_B
4_A
4_B
5_A
5_B
6_A
6_B
7_A
7_B
8_A
8_B
9_A
9_B
0_A
0_B
1_A
1_B
2_A
2_B
3_A
3_B
4_A
4_B
5_A
5_B
6_A
6_B
7_A
7_B
8_A
8_B
9_A
9_B


In [26]:
all_records_disfluency_dataframe.to_csv('1-19_all_records_disfluency_labels_with_count.csv')
all_records_fluent_dataframe.to_csv('1-19_all_records_fluent_labels_with_count.csv')

In [35]:
all_records_dataframe

,ParticipantID,Disfluency,EventNo,Wrist_X,Wrist_Y
0,1_A,1,1,0.244067,0.370307
1,1_A,1,1,0.241562,0.360925
2,1_A,1,1,0.244293,0.355284
3,1_A,1,1,0.258994,0.380093
4,1_A,1,1,0.258011,0.370002
...,...,...,...,...,...
137139,9_B,0,218,0.801131,0.701714
137140,9_B,0,218,0.801131,0.701714
137141,9_B,0,218,0.801131,0.701714
137142,9_B,0,218,0.801131,0.701714


In [ ]:
def find_partner(participants, me):
    for p in participants:
        if p['session_id'] == me['session_id'] and p['participant_id'] != me['participant_id']:
            return p

In [24]:
### listeners and speakers 

###  + just the listeners

all_records_listener2 = []

all_records_listener_only2 = []
#all_records_listener_only = all_records_listener_only.to_frame()

for rows in dict_df.itertuples():
    session_id = str(rows.Index)
    print('session')
    print(session_id)
    participant_id = rows.participant_id
    
    # print(participant) # contains all dictionary of the participant
    print('speaker')
    print(participant_id)
    
    mp_session = sessions_mp_dataframe[(sessions_mp_dataframe['Session'] == session_id)]

    mp_participant = sessions_mp_dataframe[(sessions_mp_dataframe['Participant'] == participant_id)]

    mp_listener = mp_session[~mp_session.Participant.isin([participant_id])] # yes, listener!!!!
    
    mp_times = mp_participant['Frame']

    partials = rows.partials
    fluents = rows.fluents
    
    for partial in partials:
        start = partials[partial]['start_time']
        end = partials[partial]['end_time']
        partial_no = partials[partial]['partial_no']
        for index, row in mp_participant.iterrows():
            time = row['Frame']
            hand_position_X = row['Wrist_X']
            hand_position_Y = row['Wrist_Y']   
            if start <= time and time <= end:
                disfluency = 1
                mp_participant_partial_handX = hand_position_X 
                mp_participant_partial_handY = hand_position_Y
                
                mp_listener_partial = mp_listener[(mp_listener['Frame'] == time)]
                
                mp_listener_partial_handX = mp_listener[mp_listener['Frame'] == time]['Wrist_X']
                mp_listener_partial_handY = mp_listener[mp_listener['Frame'] == time]['Wrist_Y']
                
                # mp_listener_partial_handX_mean = mp_listener_partial_handX.mean(skipna=True)
                # mp_listener_partial_handY_mean = mp_listener_partial_handY.mean(skipna=True)
                
                # separate listeners do it with for listener_time in mp_listener then append
                all_records_listener2.append([participant_id, disfluency, partial_no, mp_participant_partial_handX, mp_participant_partial_handY, mp_listener_partial_handX, mp_listener_partial_handY])
                
                for a, instances in mp_listener_partial.iterrows():
                    # print(instances)
                    listener_id = mp_listener['Participant'].values[1]
                    # print('listener')
                    # print(listener_id)

                    listener_X = instances['Wrist_X']
                    listener_Y = instances['Wrist_Y']
                    
                    all_records_listener_only2.append([listener_id, participant_id, disfluency, partial_no, listener_X, listener_Y])
         
    for fluent in fluents:
        start2 = fluents[fluent]['start_time']
        end2 = fluents[fluent]['end_time']
        fluent_no = fluents[fluent]['fluent_no']
        for index2, row2 in mp_participant.iterrows():
            time2 = row2['Frame']
            hand_position_X2 = row2['Wrist_X']
            hand_position_Y2 = row2['Wrist_Y'] 
            if start2 <= time2 and time2 <= end2:           
                disfluency = 0
                mp_participant_fluent_handX = hand_position_X2
                mp_participant_fluent_handY = hand_position_Y2
                
                mp_listener_fluent = mp_listener[mp_listener['Frame'] == time2]
                mp_listener_fluent_handX = mp_listener[mp_listener['Frame'] == time2]['Wrist_X']
                mp_listener_fluent_handY = mp_listener[mp_listener['Frame'] == time2]['Wrist_Y']
                
                mp_listener_fluent_handX_mean = mp_listener_fluent_handX.mean(skipna=True)
                mp_listener_fluent_handY_mean = mp_listener_fluent_handY.mean(skipna=True)
                
                all_records_listener2.append([participant_id, disfluency, fluent_no, mp_participant_fluent_handX, mp_participant_fluent_handY, mp_listener_fluent_handX, mp_listener_fluent_handY]) 

                for b, instances2 in mp_listener_fluent.iterrows():
                    
                    listener_flu_X = instances2['Wrist_X']
                    listener_flu_Y = instances2['Wrist_Y']
                    all_records_listener_only2.append([listener_id, participant_id, disfluency, fluent_no, listener_flu_X, listener_flu_Y])
                
all_records_dataframe2 = pd.DataFrame(all_records_listener2, columns = ['ParticipantID', 'Disfluency', 'EventNo', 'Wrist_X', 'Wrist_Y', 'Listener_Wrist_X', 'Listener_Wrist_Y'])
          
all_records_listener_only_dataframe2 = pd.DataFrame(all_records_listener_only2, columns = ['ListenerID', 'ParticipantID', 'Disfluency', 'EventNo', 'Listener_Wrist_X', 'Listener_Wrist_Y'])

session
1
speaker
1_A
session
1
speaker
1_B
session
2
speaker
2_A
session
2
speaker
2_B
session
3
speaker
3_A
session
3
speaker
3_B
session
4
speaker
4_A
session
4
speaker
4_B
session
5
speaker
5_A
session
5
speaker
5_B
session
6
speaker
6_A
session
6
speaker
6_B
session
7
speaker
7_A
session
7
speaker
7_B
session
8
speaker
8_A
session
8
speaker
8_B
session
9
speaker
9_A
session
9
speaker
9_B


In [23]:
all_records_dataframe.to_csv('all_records_listener_speaker_2.csv')
all_records_listener_only_dataframe.to_csv('all_records_listener_only.csv')

In [27]:
all_records_dataframe_dropped = all_records_dataframe.dropna()

In [29]:
all_records_dataframe_dropped.to_csv('all_records_listener_speaker_dropped.csv')

In [8]:
inter = sessions_mp_dataframe[(sessions_mp_dataframe['Participant'] == '1_A')]
inter

,Frame,Wrist_X,Wrist_Y,Participant,Session
0,0.00,0.260420,0.418896,1_A,1
1,0.04,0.252418,0.576465,1_A,1
2,0.08,0.260182,0.420576,1_A,1
3,0.08,0.252568,0.576337,1_A,1
4,0.12,0.253143,0.576375,1_A,1
...,...,...,...,...,...
50108,1631.76,0.252350,0.587557,1_A,1
50109,1631.80,0.267880,0.453328,1_A,1
50110,1631.80,0.252214,0.587440,1_A,1
50111,1631.84,0.252288,0.587518,1_A,1
